<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap10_create_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import load_dataset

# GLUE からMNLIデータセットを読み込む
# 0 = 合意、１ = 中立、２ = 矛盾
train_dataset = load_dataset(
    "glue", "mnli", split="train"
).select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mnli/train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

mnli/validation_matched-00000-of-00001.p(…):   0%|          | 0.00/1.21M [00:00<?, ?B/s]

mnli/validation_mismatched-00000-of-0000(…):   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [2]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

In [3]:
from sentence_transformers import SentenceTransformer

# ベースモデルを使用する
embedding_model = SentenceTransformer("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
from sentence_transformers import losses

# 損失関数を定義する。ソフトマックス損失では、ラベルの数を明示的に指定する必要がある
train_loss = losses.SoftmaxLoss(
    model=embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [6]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB 用に埋め込み類似性評価機を作成する
val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score / 5 for score in val_sts["label"]],
    main_similarity="cosine",
)

stsb/train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

stsb/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

stsb/test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [7]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 学習用の引数を定義する
args = SentenceTransformerTrainingArguments(
    output_dir="base_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to="none",
)

In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer

# 埋め込みモデルを学習する
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

In [ ]:
# 学習したモデルを評価する
evaluator(embedding_model)

In [ ]:
from mteb import MTEB

# 評価タスクの選択
evaluation = MTEB(task=["Banking77Classification"])

# 評価の実行
results = evaluation.run(embedding_model)

results

In [ ]:
from datasets import Dataset, load_datset

# GLUE からMNLIデータセットを読み込む
train_dataset = load_dataset(
    "glue", "mnli", split="train"
)select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

# 中立と矛盾のラベルを0、含意を１に変換する
mapping = {2: 0, 1: 0, 0:1}
train_dataset = Dataset.from_dict({
  "sentence1": train_dataset["premise"],
  "sentence2": train_dataset["hypothesis"]
  "label": [float(mapping[label]) for label in train_dataset["label"]]
})

In [ ]:
from sentense_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB 用の埋め込み類似性評価機を作成する
val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score / 5 for score in val_sts["label"]],
    main_similarity="cosine",
)